# day 1 Installation and basic operations on columns

In [5]:
!pip install pyspark

ERROR: Invalid requirement: 'pyspark#'


In [6]:
from pyspark.sql import SparkSession

In [7]:
spark=SparkSession.builder.appName('Dataframe').getOrCreate()

22/06/29 20:54:57 WARN Utils: Your hostname, Gotham.local resolves to a loopback address: 127.0.0.1; using 192.168.1.75 instead (on interface en0)
22/06/29 20:54:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/06/29 20:54:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
spark

In [9]:
#read the dataset
df_pyspark=spark.read.option('header','true').csv('test1.csv')

In [10]:
#check schema
df_pyspark.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- experience: string (nullable = true)



In [11]:
df_pyspark=spark.read.option('header','true').csv('test1.csv', inferSchema=True)

In [12]:
df_pyspark.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: integer (nullable = true)



In [13]:
#another way of reading
df_pyspark=spark.read.csv('test1.csv',header=True,inferSchema=True)

In [14]:
df_pyspark.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: integer (nullable = true)



In [15]:
df_pyspark.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: integer (nullable = true)



In [16]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [17]:
#select a column

In [18]:
df_pyspark.select('Name')

DataFrame[Name: string]

In [19]:
df_pyspark.select('Name').show()

+---------+
|     Name|
+---------+
|    krish|
|sudhanshu|
|    sunny|
+---------+



In [20]:
#to see list of all columns

df_pyspark.columns

['name', 'age', 'experience']

In [21]:
#check datatypes
df_pyspark.dtypes

[('name', 'string'), ('age', 'int'), ('experience', 'int')]

In [22]:
df_pyspark.describe().show()

+-------+-----+----+-----------------+
|summary| name| age|       experience|
+-------+-----+----+-----------------+
|  count|    3|   3|                3|
|   mean| null|30.0|7.333333333333333|
| stddev| null| 1.0|3.055050463303893|
|    min|krish|  29|                4|
|    max|sunny|  31|               10|
+-------+-----+----+-----------------+



In [23]:
#adding columns to dataframe
df_pyspark=df_pyspark.withColumn('Experience after 2 years', df_pyspark['experience']+2)

In [24]:
df_pyspark.show()

+---------+---+----------+------------------------+
|     name|age|experience|Experience after 2 years|
+---------+---+----------+------------------------+
|    krish| 31|        10|                      12|
|sudhanshu| 30|         8|                      10|
|    sunny| 29|         4|                       6|
+---------+---+----------+------------------------+



In [25]:
#dropping the columns 
df_pyspark=df_pyspark.drop('Experience after 2 years')

In [26]:
df_pyspark.show()

+---------+---+----------+
|     name|age|experience|
+---------+---+----------+
|    krish| 31|        10|
|sudhanshu| 30|         8|
|    sunny| 29|         4|
+---------+---+----------+



In [27]:
#rename the columns

In [28]:
df_pyspark=df_pyspark.withColumnRenamed('Name','NewName')

In [29]:
df_pyspark.show()

+---------+---+----------+
|  NewName|age|experience|
+---------+---+----------+
|    krish| 31|        10|
|sudhanshu| 30|         8|
|    sunny| 29|         4|
+---------+---+----------+



# day2 Handle missing value by mean, median etc.

In [30]:
df_pyspark=spark.read.csv('test2.csv',header=True,inferSchema=True)

In [31]:
df_pyspark.show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|null|      null| 40000|
|     null|  34|        10| 38000|
|     null|  36|      null|  null|
+---------+----+----------+------+



In [33]:
#drop null values 

df_pyspark.na.drop().show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [34]:
# in above cell all the rows which has atleast one null column gets deleted. 

In [35]:
#understanding features of drop() : how, thresh and subset
df_pyspark.na.drop(how='all').show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|null|      null| 40000|
|     null|  34|        10| 38000|
|     null|  36|      null|  null|
+---------+----+----------+------+



In [37]:
#in above cell, no rows where dropped because when we select how=all, it looks for rows which have all fields null. By default how is any


In [38]:
#threshold
df_pyspark.na.drop(thresh=2).show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|null|      null| 40000|
|     null|  34|        10| 38000|
+---------+----+----------+------+



In [39]:
# at least 2 non-null value should be present. i.e the rows having more than two non-null values will be dropped.

In [40]:
#subset
df_pyspark.na.drop(subset=['Experience']).show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
|     null| 34|        10| 38000|
+---------+---+----------+------+



In [ ]:
# wherever the column Experience was null, the corresponding row is dropped.  

### fill missing value

In [41]:
# lets start with the original dataset first
df_pyspark.show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|null|      null| 40000|
|     null|  34|        10| 38000|
|     null|  36|      null|  null|
+---------+----+----------+------+



In [51]:
# fill has two features: value and subset
#lets look into value 

df_pyspark.na.fill('Missing').show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|null|      null| 40000|
|  Missing|  34|        10| 38000|
|  Missing|  36|      null|  null|
+---------+----+----------+------+



In [ ]:
#this filled 'Missing' wherever the field of type string is null.

In [59]:
df_pyspark.na.fill(999).show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
|   Mahesh|999|       999| 40000|
|     null| 34|        10| 38000|
|     null| 36|       999|   999|
+---------+---+----------+------+



In [60]:
#similarly the above cell filled 999 wherever the field of type integer is null.

In [61]:
# if we want to fill all the cells with either 'Missing' or 999, we have to load the dataset with feature inferSchema=False


In [63]:
# filling missing value only on specific columns
df_pyspark.na.fill(999,['Experience','Salary']).show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|null|       999| 40000|
|     null|  34|        10| 38000|
|     null|  36|       999|   999|
+---------+----+----------+------+



### filling with mean 

In [64]:
from pyspark.ml.feature import Imputer

In [71]:
imputer= Imputer(
inputCols=['age','Experience','Salary'],outputCols=[x+'_imp' for x in ['age','Experience','Salary']]).setStrategy('mean')


In [72]:
imputer.fit(df_pyspark).transform(df_pyspark).show()

+---------+----+----------+------+-------+--------------+----------+
|     Name| age|Experience|Salary|age_imp|Experience_imp|Salary_imp|
+---------+----+----------+------+-------+--------------+----------+
|    Krish|  31|        10| 30000|     31|            10|     30000|
|Sudhanshu|  30|         8| 25000|     30|             8|     25000|
|    Sunny|  29|         4| 20000|     29|             4|     20000|
|     Paul|  24|         3| 20000|     24|             3|     20000|
|   Harsha|  21|         1| 15000|     21|             1|     15000|
|  Shubham|  23|         2| 18000|     23|             2|     18000|
|   Mahesh|null|      null| 40000|     28|             5|     40000|
|     null|  34|        10| 38000|     34|            10|     38000|
|     null|  36|      null|  null|     36|             5|     25750|
+---------+----+----------+------+-------+--------------+----------+



# day 3 Pyspark Dataframes: Filter, Aggregation Operations 

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('dataframe').getOrCreate()

22/07/04 21:25:18 WARN Utils: Your hostname, Gotham.local resolves to a loopback address: 127.0.0.1; using 192.168.1.75 instead (on interface en0)
22/07/04 21:25:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/04 21:25:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_pyspark=spark.read.csv('test1.csv',header=True,inferSchema=True)

In [4]:
df_pyspark.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [5]:
# filtering those who have salary less than or equal to 20K
df_pyspark.filter("Salary<=20000").show()

+-------+---+----------+------+
|   Name|age|Experience|Salary|
+-------+---+----------+------+
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
+-------+---+----------+------+



In [6]:
# picking just Name and Experience of those who have salary less than or equal to 20K 
df_pyspark.filter("Salary<=20000").select(['Name','Experience']).show()

+-------+----------+
|   Name|Experience|
+-------+----------+
|  Sunny|         4|
|   Paul|         3|
| Harsha|         1|
|Shubham|         2|
+-------+----------+



In [7]:
# filtering based on multiple conditions
df_pyspark.filter((df_pyspark["Salary"]<=20000) & (df_pyspark["Experience"]>2)).show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
|Sunny| 29|         4| 20000|
| Paul| 24|         3| 20000|
+-----+---+----------+------+



In [8]:
# using negation (~) in filter condition
df_pyspark.filter(~(df_pyspark["Salary"]<=20000) & (df_pyspark["Experience"]>2)).show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
+---------+---+----------+------+



# Groupby and Aggregation Functions

In [9]:
df_pyspark=spark.read.csv('test3.csv',header=True, inferSchema=True)

In [10]:
df_pyspark.show()

+---------+------------+------+
|     Name| Departments|salary|
+---------+------------+------+
|    Krish|Data Science| 10000|
|    Krish|         IOT|  5000|
|   Mahesh|    Big Data|  4000|
|    Krish|    Big Data|  4000|
|   Mahesh|Data Science|  3000|
|Sudhanshu|Data Science| 20000|
|Sudhanshu|         IOT| 10000|
|Sudhanshu|    Big Data|  5000|
|    Sunny|Data Science| 10000|
|    Sunny|    Big Data|  2000|
+---------+------------+------+



In [11]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Departments: string (nullable = true)
 |-- salary: integer (nullable = true)



### Groupby

In [12]:
df_pyspark.groupBy('Name').sum()

DataFrame[Name: string, sum(salary): bigint]

In [13]:
df_pyspark.groupBy('Name').sum().show()

+---------+-----------+
|     Name|sum(salary)|
+---------+-----------+
|Sudhanshu|      35000|
|    Sunny|      12000|
|    Krish|      19000|
|   Mahesh|       7000|
+---------+-----------+



In [14]:
df_pyspark.groupBy('Departments').mean().show()

+------------+-----------+
| Departments|avg(salary)|
+------------+-----------+
|         IOT|     7500.0|
|    Big Data|     3750.0|
|Data Science|    10750.0|
+------------+-----------+



In [15]:
df_pyspark.groupBy('Departments').count().show()

+------------+-----+
| Departments|count|
+------------+-----+
|         IOT|    2|
|    Big Data|    4|
|Data Science|    4|
+------------+-----+



# Linear Regression Basic Example

In [108]:
spark=SparkSession.builder.appName("lin_reg").getOrCreate()

In [109]:
df_pyspark_reg=spark.read.csv('tips.csv',header=True,inferSchema=True)

In [110]:
df_pyspark_reg.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [126]:
df_pyspark_reg.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [112]:
# we dont need to fill. issing value here. Lets directly jump to Linear Regression.

In [113]:
#in order to apply linear regeression, we must select dependent and independent feature. 

In [114]:
# here "total_bill" is our target(dependent) variable and rest are independent variables 

In [115]:
# lets convert the strings features to numerical features wherever possible

### impute

In [127]:
from pyspark.ml.feature import StringIndexer

In [128]:
# sex , smoker , day , time

In [129]:
indexer=StringIndexer(inputCols=['sex','smoker','day','time'], outputCols=[i+'_transform' for i in (['sex','smoker','day','time'])])


In [148]:
df_pyspark_reg_indexed=indexer.fit(df_pyspark_reg).transform(df_pyspark_reg).show()

+----------+----+------+------+---+------+----+-------------+----------------+-------------+--------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_transform|smoker_transform|day_transform|time_transform|
+----------+----+------+------+---+------+----+-------------+----------------+-------------+--------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|          1.0|             0.0|          1.0|           0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|          0.0|             0.0|          1.0|           0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|          0.0|             0.0|          1.0|           0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|          0.0|             0.0|          1.0|           0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|          1.0|             0.0|          1.0|           0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|          0.0|             0.0|          1.0|           0.0|
|      8.77| 2.0|  Male|    

In [149]:
# from here, we can drop the unnecessary columns but we will simoly use something Interesting such as vector assembler


In [150]:
from pyspark.ml.feature import VectorAssembler

In [151]:
# df_pyspark_reg.select(df_pyspark_reg.columns[1]).show()

In [152]:
features=VectorAssembler(inputCols=['tip','size','sex_transform','smoker_transform','day_transform','time_transform'], outputCol='Vector_Features')







In [153]:
output=features.transform(df_pyspark_reg_indexed)

AttributeError: 'NoneType' object has no attribute '_jdf'

In [155]:
# This 'NoneType' object has no attribute '_jdf' error is because somehwere in assign line of code we must have used show()

# Just remove show and it will work fine. I have the same issue in line 148. 


In [156]:
df_pyspark_reg_indexed=indexer.fit(df_pyspark_reg).transform(df_pyspark_reg)

In [157]:
df_pyspark_reg_indexed.show()

+----------+----+------+------+---+------+----+-------------+----------------+-------------+--------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_transform|smoker_transform|day_transform|time_transform|
+----------+----+------+------+---+------+----+-------------+----------------+-------------+--------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|          1.0|             0.0|          1.0|           0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|          0.0|             0.0|          1.0|           0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|          0.0|             0.0|          1.0|           0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|          0.0|             0.0|          1.0|           0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|          1.0|             0.0|          1.0|           0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|          0.0|             0.0|          1.0|           0.0|
|      8.77| 2.0|  Male|    

In [ ]:
# now i can use VectorAssembler

In [158]:
output=features.transform(df_pyspark_reg_indexed)

In [159]:
output.show()

+----------+----+------+------+---+------+----+-------------+----------------+-------------+--------------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_transform|smoker_transform|day_transform|time_transform|     Vector_Features|
+----------+----+------+------+---+------+----+-------------+----------------+-------------+--------------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|          1.0|             0.0|          1.0|           0.0|[1.01,2.0,1.0,0.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|          0.0|             0.0|          1.0|           0.0|[1.66,3.0,0.0,0.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|          0.0|             0.0|          1.0|           0.0|[3.5,3.0,0.0,0.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|          0.0|             0.0|          1.0|           0.0|[3.31,2.0,0.0,0.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|          1.0|             0.0|          1.0|     

In [161]:
output.select('Vector_Features').show()

+--------------------+
|     Vector_Features|
+--------------------+
|[1.01,2.0,1.0,0.0...|
|[1.66,3.0,0.0,0.0...|
|[3.5,3.0,0.0,0.0,...|
|[3.31,2.0,0.0,0.0...|
|[3.61,4.0,1.0,0.0...|
|[4.71,4.0,0.0,0.0...|
|[2.0,2.0,0.0,0.0,...|
|[3.12,4.0,0.0,0.0...|
|[1.96,2.0,0.0,0.0...|
|[3.23,2.0,0.0,0.0...|
|[1.71,2.0,0.0,0.0...|
|[5.0,4.0,1.0,0.0,...|
|[1.57,2.0,0.0,0.0...|
|[3.0,4.0,0.0,0.0,...|
|[3.02,2.0,1.0,0.0...|
|[3.92,2.0,0.0,0.0...|
|[1.67,3.0,1.0,0.0...|
|[3.71,3.0,0.0,0.0...|
|[3.5,3.0,1.0,0.0,...|
|(6,[0,1],[3.35,3.0])|
+--------------------+
only showing top 20 rows



In [162]:
# prepare final dataset

In [163]:
final_dataset=output.select('Vector_Features','total_bill')

In [164]:
final_dataset.show()

+--------------------+----------+
|     Vector_Features|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
|[3.5,3.0,0.0,0.0,...|     21.01|
|[3.31,2.0,0.0,0.0...|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|[4.71,4.0,0.0,0.0...|     25.29|
|[2.0,2.0,0.0,0.0,...|      8.77|
|[3.12,4.0,0.0,0.0...|     26.88|
|[1.96,2.0,0.0,0.0...|     15.04|
|[3.23,2.0,0.0,0.0...|     14.78|
|[1.71,2.0,0.0,0.0...|     10.27|
|[5.0,4.0,1.0,0.0,...|     35.26|
|[1.57,2.0,0.0,0.0...|     15.42|
|[3.0,4.0,0.0,0.0,...|     18.43|
|[3.02,2.0,1.0,0.0...|     14.83|
|[3.92,2.0,0.0,0.0...|     21.58|
|[1.67,3.0,1.0,0.0...|     10.33|
|[3.71,3.0,0.0,0.0...|     16.29|
|[3.5,3.0,1.0,0.0,...|     16.97|
|(6,[0,1],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [165]:
#  applying linear regression

In [176]:
from pyspark.ml.regression import LinearRegression

In [184]:
train_data, test_data=final_dataset.randomSplit([0.75,0.25])

In [185]:
lr=LinearRegression(featuresCol='Vector_Features',labelCol='total_bill')

In [186]:
x=lr.fit(train_data)

22/07/04 23:36:32 WARN Instrumentation: [9b9e2314] regParam is zero, which might cause numerical instability and overfitting.


In [187]:
x.coefficients

DenseVector([3.0826, 3.8006, -1.0978, 2.1781, -0.5032, 0.2479])

In [188]:
x.intercept

0.7990600360466894

In [189]:
# prediction part

In [192]:
pred=x.evaluate(test_data)

In [194]:
pred.predictions.show()

+--------------------+----------+------------------+
|     Vector_Features|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[1.25,2.0])|     10.51| 12.25350913025802|
|(6,[0,1],[1.45,2.0])|      9.55|12.870029620801512|
|(6,[0,1],[1.47,2.0])|     10.77|12.931681669855863|
|(6,[0,1],[1.97,2.0])|     12.02|14.472982896214592|
| (6,[0,1],[2.0,2.0])|     13.37|14.565460969796117|
|(6,[0,1],[2.01,2.0])|     20.23| 14.59628699432329|
| (6,[0,1],[2.5,4.0])|     18.35| 23.70795822446935|
| (6,[0,1],[3.0,4.0])|     20.45| 25.24925945082808|
|(6,[0,1],[3.15,3.0])|     20.08| 21.91105180457845|
|(6,[0,1],[3.27,2.0])|     17.78|18.480366084747292|
| (6,[0,1],[3.6,3.0])|     24.06|23.298222908301305|
|(6,[0,1],[5.92,3.0])|     29.03|30.449860598605817|
|(6,[0,1],[6.73,4.0])|     48.27|36.747366599464215|
| (6,[0,1],[9.0,4.0])|     48.33| 43.74487416713285|
|[1.5,2.0,0.0,0.0,...|     19.08|12.265575816245715|
|[1.5,2.0,0.0,1.0,...|     11.59|15.2023015007

In [197]:
# prediction metrics
pred.r2, pred.meanAbsoluteError, pred.meanSquaredError

(0.5229535456102288, 4.386360919202124, 35.3419878455901)